# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [ ]:
pip install gspread

In [ ]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [ ]:
ls

In [ ]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [ ]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [ ]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']
# print(user_dat)

### TODO #1
Hoàn tất đoạn code sau

In [ ]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])
print(user_dict)

In [ ]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(10) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

In [ ]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [ ]:
!ls

In [ ]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'speedy-precept-314204-30d59f2d7609.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [ ]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1b9XndW1uqs1ZS7c_Qre1JuT7OnGfumguaBN4IxXS46Y' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [179]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']
print(channel_ls)

[{'id': 'C01B4PVGLVB', 'name': 'general', 'is_channel': True, 'is_group': False, 'is_im': False, 'created': 1600856703, 'is_archived': False, 'is_general': True, 'unlinked': 0, 'name_normalized': 'general', 'is_shared': False, 'parent_conversation': None, 'creator': 'U01BE2PR6LU', 'is_ext_shared': False, 'is_org_shared': False, 'shared_team_ids': ['T01B7SGGMLJ'], 'pending_shared': [], 'pending_connected_team_ids': [], 'is_pending_ext_shared': False, 'is_member': False, 'is_private': False, 'is_mpim': False, 'topic': {'value': '', 'creator': '', 'last_set': 0}, 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.', 'creator': 'U01BE2PR6LU', 'last_set': 1600856703}, 'previous_names': [], 'num_members': 63}, {'id': 'C01BYH7JHB5', 'name': 'contents', 'is_channel': True, 'is_group': False, 'is_im': False, 'created': 1601882889, 'is_archived': False, 'is_general': False, 'unlinked': 0, 'name_normaliz

In [197]:
find_ID_channel={"Channels":[]}
for i in channel_ls:
    if i['name'] == 'atom-assignment2':
        find_ID_channel['Channels'].append(i['id'])

print(find_ID_channel)

{'Channels': ['C021FSDN7LJ']}


### TODO#3 
* Tìm id của channel #atom-assignment2

### Slack API: List messages trong 1 channel

In [198]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#2 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [199]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [212]:
msg_ls[20]

{'client_msg_id': '12a69b88-ca3b-4d80-a9ac-b7a25e46cfb9',
 'type': 'message',
 'text': 'Chào mọi người, em xin gửi assignment 2 nhé ạ.:\n<https://github.com/hoaintp/datacracy-atom-hoaintp>',
 'user': 'U01UTGVPE7N',
 'ts': '1621016067.022900',
 'team': 'T01B7SGGMLJ',
 'attachments': [{'fallback': 'hoaintp/datacracy-atom-hoaintp',
   'title': 'hoaintp/datacracy-atom-hoaintp',
   'id': 1,
   'color': '24292f',
   'fields': [{'title': 'Language',
     'value': 'Jupyter Notebook',
     'short': True},
    {'title': 'Last updated', 'value': '6 minutes ago', 'short': True}],
   'bot_id': 'B01UQU8E2A3',
   'app_unfurl_url': 'https://github.com/hoaintp/datacracy-atom-hoaintp',
   'is_app_unfurl': True}],
 'blocks': [{'type': 'rich_text',
   'block_id': 'VtoIv',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text',
       'text': 'Chào mọi người, em xin gửi assignment 2 nhé ạ.:\n'},
      {'type': 'link',
       'url': 'https://github.com/hoaintp/datacracy-atom-hoaintp

In [201]:
not_learners_id = ['U01BE2PR6LU']

In [220]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
list_done={"time":[],"name":[],"teams":[],"Link":[]}
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  list_done["time"].append(ts)
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  list_done["name"].append(user)
  _2 =msg_ls[i]['team']
  list_done['teams'].append(_2)
  if msg_ls[i]['user'] not in not_learners_id:
      _file = msg_ls[i]['attachments']
      list_done['Link'].append(_file)

  print(list_done)


KeyError: 'team'

In [158]:
user_df = pd.DataFrame(member) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

ValueError: arrays must all be same length

### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

-------------- HẾT PHẦN OPTION 2 ---------------------